# Universidade Federal do ABC - UFABC
## Centro de Matemática, Computação e Cognição - CMCC

## Disciplina: Visão Computacional e Processamento de Imagens

Responsável: Prof. Dr. Francisco Zampirolli

Estudante: [Bruno Aristimunha](https://github.com/bruAristimunha).

Santo André, Terceiro Quadrimestre de 2019

### Projeto Final da Disciplina

#### Classificação de Grão de Pólen.

## Definição do Problema

Os grãos de pólen são importantes marcadores geológicos e geográficos presentes em todo o globo. Suas aplicações são diversas, mas dentre as mais comuns podemos citar o uso para a perícia investigativa, o mapeamento do clima em função de milhares de anos e estudos alérgicos, além da produção de alimentos à base de mel. Em todas as áreas citadas, para que se obtenha resultados significativos de análises robustas, faz-se necessário o levantamento estatístico da distribuição dos tipos de pólen presentes em uma amostra.

Considerando a diversidade, as semelhanças interespécies e características microscópicas, o reconhecimento de cada espécie demanda uma ampla e longa formação botânica. O processo de aquisição está exposto a seguir na Figura 1.
![aquisicao](https://raw.githubusercontent.com/bruAristimunha/pollenData/master/figs/capture-pipeline.png)

#### Figura 01: Processo de aquisição dos grãos de pólen. Inicialmente a amostra é capturada em uma lâmina, há ampliação da imagem no microscópio posteriormente, busca-se focalizar o grão e estudar sua morfologia, classificando a espécie.

O processo não trivial de aquisição da imagem demanda tempo, custos em materiais, além de técnica e anos de experiência para classificação. Ademais, dependendo do material usado para aquisição pode haver visualizações distintas para a mesma espécie de pólen. A Figura 2 mostra a diferença entre metodologias de captura de imagens.

In [1]:
from skimage import io
import matplotlib.pyplot as plt
url1 = "https://raw.githubusercontent.com/bruAristimunha/pollenData/master/figs/solvent2.png"
url2 = "https://raw.githubusercontent.com/bruAristimunha/pollenData/master/figs/solvent1.png"

fig = plt.figure(figsize=(20,20))

image1 = io.imread(url1)
image2 = io.imread(url2)

plt.subplot(221).imshow(image1)
plt.subplot(222).imshow(image2)
plt.show()


<Figure size 2000x2000 with 2 Axes>

#### Figura 02: A diferença entre solventes gera uma coloração totalmente distinta quando visualizada no microscópio, empecilhos como esses dificultam a generalização da classificação do biólogo.


O aprendizado profundo é o conjunto de técnicas comumente utilizadas em visão computacional para busca de
representações hierárquicas de dados. Métodos de camadas convolucionais, permitem solucionar inúmeros problemas da visão computacional pela sua invariância translacional e conectividade local. Em outras palavras, empregamos esse conjunto de técnicas para buscar padrões de formação desconhecidos a priori pelo classificador, que só será descoberto durante o aprendizado.

## Dados/imagens

Será utilizado um banco de imagens de polens nativos do Mato Grosso do Sul. Por se tratar de um objeto de geometria tridimensional, as diferentes angulações influenciam tanto na classificação do especialista quanto em algoritmos tradicionais de classificação, portanto cada grão possui 35 imagens distintas.

O conjunto de dados possui [23](http://palinovic.weebly.com/bancos-de-imagens.html) classes balanceadas, com um total de 805 imagens. Técnicas de aumento de dados nas imagens serão largamente empregadas. Optou-se pelo aumento de dados empregando rotações de 45º, traçando um paralelo com a literatura botânica, que emprega rotações de 45º na captura de imagens. Por se tratar de um objeto de geometria tridimensional levantou-se a hipótese de que essas rotações auxiliariam na generalização dos métodos. Será considerada a divisão: treino, validação e teste, sendo 60, 20 e 20 a porcentagem de cada uma de acordo com a quantidade de imagens. 


## Processamento de imagens


In [2]:
per_train = 0.6
per_test = 0.2
per_validation = 0.2
seed_split_folder = 42


#model
img_width, img_height = 299, 299
train_data_dir = "../data/pollen23e_split/train"
validation_data_dir = "../data/pollen23e_split/validation"
batch_size = 16
epochs = 100

lr=0.0001
momentum=0.9

#indice do modelo testado
indice = 3

Nesse primeiro momento realizando uma separação do conjunto de dados em 

In [3]:
import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio('../data/pollen23e/', output="../data/pollen23e_split", seed=seed_split_folder, ratio=(per_train, per_test, per_validation)) # default values


Copying files: 801 files [00:00, 5111.12 files/s]


In [4]:
#Bibliotecas empregadas na análise

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [5]:
#Biblioteca para impressão do versionamento das bibliotecas importadas. Importante para reprodutibilidade do trabalho
%load_ext watermark
print("\nSistema\n")
%watermark
print("\nBibliotecas\n")
%watermark --iversions


Sistema

2019-11-06T19:46:01-03:00

CPython 3.6.8
IPython 5.5.0

compiler   : GCC 8.3.0
system     : Linux
release    : 5.0.0-32-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit

Bibliotecas

skimage         0.16.2
IPython         5.5.0
matplotlib      3.1.1
tensorflow_core 2.0.0



In [6]:
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201, InceptionResNetV2, InceptionV3, MobileNet, MobileNetV2, NASNetLarge, NASNetMobile, ResNet101, ResNet101V2, ResNet152, ResNet152V2, ResNet50, ResNet50V2, VGG16, VGG19, Xception

from tensorflow.keras.layers import Dense

models = [DenseNet121, DenseNet169, DenseNet201, InceptionResNetV2, InceptionV3, MobileNet, MobileNetV2, NASNetLarge, NASNetMobile, ResNet101, ResNet101V2, ResNet152, ResNet152V2, ResNet50, ResNet50V2, VGG16, VGG19, Xception]
models_name =  ['DenseNet121','DenseNet169','DenseNet201','InceptionResNetV2','InceptionV3','MobileNet','MobileNetV2','NASNetLarge','NASNetMobile','ResNet101','ResNet101V2','ResNet152','ResNet152V2','ResNet50','ResNet50V2','VGG16','VGG19','Xception']


In [7]:
model = models[indice](weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#Adding custom Layers
x = model.output
x = Flatten()(x)
predictions = Dense(23, activation="softmax",use_bias=False)(x)
# Creating the final model
model_final = Model(model.input,predictions)
# compile the model
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=lr, momentum=momentum), metrics=["accuracy"])


In [8]:
train_datagen = ImageDataGenerator(
           rotation_range=45,
           width_shift_range=0.1,
           height_shift_range=0.1,
           shear_range=0.01,
           zoom_range=[0.9,1.25],
           horizontal_flip=True,
           vertical_flip=True,
           fill_mode='reflect',
           data_format='channels_last',
           brightness_range=[0.5, 1.5])

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
                        '../data/pollen23e_split/train',
                        target_size = (img_height, img_width),
                        batch_size = batch_size,
                        save_to_dir = '../data/pollen23e_aug/train',
                        save_prefix='aug', 
                        save_format='png',
                        class_mode = "categorical",
                        seed = 45)

validation_generator = train_datagen.flow_from_directory(
                        '../data/pollen23e_split/val',
                        target_size = (img_height, img_width),
                        batch_size = batch_size,
                        class_mode = "categorical",
                        save_to_dir = '../data/pollen23e_aug/train',
                        save_prefix='aug', 
                        save_format='png',
                        seed = 45)

test_generator = test_datagen.flow_from_directory(
                        '../data/pollen23e_split/test',
                        target_size = (img_height, img_width),
                        batch_size = batch_size,
                        class_mode = 'categorical',
                        save_to_dir = '../data/pollen23e_aug/train',
                        save_prefix='aug', 
                        save_format='png',
                        seed = 45)


Found 480 images belonging to 23 classes.
Found 160 images belonging to 23 classes.
Found 161 images belonging to 23 classes.


In [9]:
# Save the model according to the conditions


checkpoint = ModelCheckpoint("checkpoints/"+models_name[indice], 
                             monitor='val_acc', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', save_freq=1)


In [ ]:
# Train the model
model_final.fit_generator(
                train_generator,
                steps_per_epoch = train_generator.samples/batch_size,
                epochs = epochs,
                callbacks = [checkpoint],
                validation_data = validation_generator,
                validation_steps = validation_generator.samples/batch_size)


Epoch 1/100


W1106 19:46:20.864938 140051437492032 callbacks.py:990] Can save best model only with val_acc available, skipping.


 1/30 [>.............................] - ETA: 4:46 - loss: 3.4349 - accuracy: 0.0625

W1106 19:46:29.094327 140051437492032 callbacks.py:990] Can save best model only with val_acc available, skipping.


 2/30 [=>............................] - ETA: 4:13 - loss: 3.3951 - accuracy: 0.0625

W1106 19:46:37.639474 140051437492032 callbacks.py:990] Can save best model only with val_acc available, skipping.


 3/30 [==>...........................] - ETA: 4:00 - loss: 3.4506 - accuracy: 0.0417

W1106 19:46:46.015789 140051437492032 callbacks.py:990] Can save best model only with val_acc available, skipping.


 4/30 [===>..........................] - ETA: 3:47 - loss: 3.3998 - accuracy: 0.0469

https://www.google.com/search?q=reprodutibility+in+computer+science+jupyter&oq=reprodutibility+in+computer+science+jupyter&aqs=chrome..69i57j33.9207j0j7&sourceid=chrome&ie=UTF-8


https://keras.io/applications/#resnet

https://github.com/jupyter-guide/ten-rules-jupyter/blob/e7b184c4949d164ce12eb5fdfbc8c8cd487b8a23/example2/0-Workflow.ipynb

https://github.com/nteract/papermill

